### raw data(중국어 상품명)로 홍보문구, 연관 키워드 생성 후 data3_base에 저장

1. data3_base.csv
    - detailed_description : 생성한 홍보문구
    - related_search_keywords : 생성한 연관 키워드 리스트
2. prompts\base에 프롬프트 텍스트 저장되어 있음

In [2]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
import pandas as pd

In [9]:
llm = ChatOpenAI(model='gpt-3.5-turbo-0125')

with open("prompts/base/product_title_prompt.txt", 'r', encoding='utf-8') as file: product_title_prompt = file.read()
with open("prompts/base/product_description_prompt.txt", 'r', encoding='utf-8') as file: product_description_prompt = file.read()
with open("prompts/base/related_keywords_prompt.txt", 'r', encoding='utf-8') as file: related_keywords_prompt = file.read()

input_prompt_text = "Rough description: {product_name} \n"

prompt1 = ChatPromptTemplate.from_template(product_title_prompt + input_prompt_text)
prompt2 = ChatPromptTemplate.from_template(product_description_prompt + input_prompt_text)
prompt3 = ChatPromptTemplate.from_template(related_keywords_prompt + input_prompt_text)

parser = StrOutputParser()


chain1 = {"product_name": RunnablePassthrough()} | prompt1 | llm | parser
chain2 = {"product_name": RunnablePassthrough()} | prompt2 | llm | parser
chain3 = {"product_name": RunnablePassthrough()} | prompt3 | llm | parser

In [11]:
data2_csv = pd.read_csv("data/data2.csv")[:11613]
product_name_kor = list(data2_csv["product_name_kor"])


product_title = chain1.batch(product_name_kor, config={"max_concurrency": 200})
detailed_description = chain2.batch(product_name_kor, config={"max_concurrency": 200})
related_keywords = chain3.batch(product_name_kor, config={"max_concurrency": 200})

In [12]:
for i in range(len(product_title)):
    if (product_title[i][:14] == "Product title:"): product_title[i] = product_title[i][15:]

for j in range(len(detailed_description)):
    if (detailed_description[j][:33] == "Detailed promotional description:"): detailed_description[j] = detailed_description[j][35:]

for k in range(len(related_keywords)):
    if ("Related search keywords:" in related_keywords[k]):
        related_keywords[k] = related_keywords[k].split("Related search keywords: ")[1][:-1]

In [13]:
df = pd.DataFrame({'product_name_kor': product_name_kor,
                   'product_title' : product_title,
                   'detailed_description': detailed_description,
                   'related_search_keywords': related_keywords})
df.to_csv("data/data3_base.csv", index=False)